# Approach

1. > compare Embeddings (```minorminer```) and choose the best
1. > compare Chain Settings and choose the best
1. > compare Anneal Times and choose the best
1. > compare Anneal Schedules and choose the best

## QUBO

### Imports

In [22]:
import sys
sys.path.append("..")
from qubo_util import *

import dimod
import numpy as np
import pandas as pd
import networkx as nx
import minorminer
import dwave.inspector
import matplotlib.pyplot as plt
from dwave.system.samplers import DWaveSampler
from dwave.embedding.chain_strength import scaled, uniform_torque_compensation
from dwave.system import DWaveSampler, FixedEmbeddingComposite, EmbeddingComposite

### Config

In [23]:
from dwave.cloud import Client
client = Client.from_config(config_file='/Users/jonas/Library/Application Support/dwave/dwave.conf')

### Jobs

In [24]:
### Lets define the basics

bend = [0, 1, 2]
weld = [3, 4]
paint =  [5]

bend_length = 2
weld_length = 3
paint_length = 6

t_step = 3
t_step_in_sec = 30

In [25]:
jobs = [(0, 0, 3, 12, 8, 20),
 (1, 0, 5, 9, 0, 10),
 (1, 1, 9, 4, 5, 10)]

In [26]:
# OrderNo, PartNo, BendingLines, WeldingPoints, PaintTime, DueDate 

m_t_steps = max_time(jobs, bend_length, weld_length, paint_length)

m_time = m_t_steps * t_step_in_sec
    
print('The maximal maketime for the given operations is: ' + str(m_time) + ' second(s).\nWhich is equal to: ' + str(m_t_steps) + ' time steps.\n\n')

The maximal maketime for the given operations is: 4530 second(s).
Which is equal to: 151 time steps.




In [27]:
operations = ops(jobs, bend, weld, paint, bend_length, weld_length, paint_length, t_step)

print('Anzahl an Kombinationen: ' + str(len(operations)) + '\n')

Anzahl an Kombinationen: 507



### Parameters

In [28]:
alpha = 2
beta = 1.5
gamma = 2
delta = 0.035

QUBO = get_QUBO(jobs, operations, alpha, beta, gamma, delta, bend_length, weld_length, paint_length, bend, weld, paint)
qubo_dictionary = qubo_to_dictionary_ohne_null(QUBO, operations)


# Run

## Get Embedding with ```  minorminer  ``` as part of EmbeddingComposite() and safe it

https://docs.dwavesys.com/docs/latest/handbook_embedding.html#example-clique-embedding-a-sparse-bqm

https://docs.dwavesys.com/docs/latest/handbook_qpu.html?highlight=sampleset_1#read-anneal-cycles

In [29]:
bqm = dimod.BinaryQuadraticModel.from_numpy_matrix(QUBO)
qpu = DWaveSampler(solver={'topology__type': 'pegasus'})

In [30]:
# Each run with 500 shots (more runs with less shots yield better results than less runs with more shots)
numr = 500

# Chain Strength = Max(QUBO) // Analog zur D Wave Dokumentation

chnstr = find_chstr(QUBO)+1

# Empty array
data=[]

### And Visualize embedding,  check for Chain Breaks and run inspection with ```dwave.inspector.```

#### First Embedding

In [31]:
solver_graph = qpu.to_networkx_graph()

In [32]:
embedding = minorminer.find_embedding(qubo_dictionary, solver_graph)

In [35]:
sampleset1A = FixedEmbeddingComposite(qpu, embedding).sample(bqm, 
                                                                   return_embedding=True, 
                                                                   answer_mode="raw", 
                                                                   num_reads=numr, 
                                                                   chain_strength= chnstr)

ValueError: no embedding found

In [34]:
######### Kann kein Embedding finden ###########
# - Weder online über EmbeddingComposite (minorminer auf D Wave), noch lokal direkt über minorminer! #

In [ ]:
analyze(sampleset1A, "embedding1A", data)

In [ ]:
sampleset1B = FixedEmbeddingComposite(qpu, embedding1).sample(bqm, 
                                                                   return_embedding=True, 
                                                                   answer_mode="raw", 
                                                                   num_reads=numr, 
                                                                   chain_strength= chnstr)

In [ ]:
analyze(sampleset1B, "embedding1B", data)

In [ ]:
print(sampleset1A.info["embedding_context"]["chain_strength"])    

In [ ]:
print(max(len(chain) for chain in chains))   

In [ ]:
print("Percentage of samples with >10% breaks is {} and >0 is {}.".format(np.count_nonzero(sampleset1A.record.chain_break_fraction > 0.10)/numr*100, np.count_nonzero(sampleset1A.record.chain_break_fraction > 0.0)/numr*100))


In [ ]:
# Inspect Embedding
dwave.inspector.show(sampleset1A)

#### Second Embedding

In [ ]:
sampler = EmbeddingComposite(qpu)
sampleset2A = sampler.sample(bqm, 
                           return_embedding=True, 
                           answer_mode="raw", 
                           num_reads=numr, 
                           chain_strength= chnstr)
embedding2 = sampleset2A.info["embedding_context"]["embedding"]  
chains = sampleset2A.info["embedding_context"]["embedding"].values()  


In [ ]:
analyze(sampleset2A, "embedding2A", data)

In [ ]:
sampleset2B = FixedEmbeddingComposite(qpu, embedding2).sample(bqm, 
                                                                   return_embedding=True, 
                                                                   answer_mode="raw", 
                                                                   num_reads=numr, 
                                                                   chain_strength= chnstr)

In [ ]:
analyze(sampleset2B, "embedding2B", data)

In [ ]:

print(sampleset2A.info["embedding_context"]["chain_strength"])    

In [ ]:
print(max(len(chain) for chain in chains))   

In [ ]:
print("Percentage of samples with >10% breaks is {} and >0 is {}.".format(np.count_nonzero(sampleset2A.record.chain_break_fraction > 0.10)/numr*100, np.count_nonzero(sampleset2A.record.chain_break_fraction > 0.0)/numr*100))


In [ ]:
# Inspect Embedding
dwave.inspector.show(sampleset2A) 

#### Third Embedding

In [ ]:
sampler = EmbeddingComposite(qpu)
sampleset3A = sampler.sample(bqm, 
                           return_embedding=True, 
                           answer_mode="raw", 
                           num_reads=numr, 
                           chain_strength= chnstr)
embedding3 = sampleset3A.info["embedding_context"]["embedding"]  
chains = sampleset3A.info["embedding_context"]["embedding"].values()  


In [ ]:
analyze(sampleset3A, "embedding3A", data)

In [ ]:
sampleset3B = FixedEmbeddingComposite(qpu, embedding3).sample(bqm, 
                                                                   return_embedding=True, 
                                                                   answer_mode="raw", 
                                                                   num_reads=numr, 
                                                                   chain_strength= chnstr)

In [ ]:
analyze(sampleset3B, "embedding3B", data)

In [ ]:

print(sampleset3A.info["embedding_context"]["chain_strength"])    

In [ ]:
print(max(len(chain) for chain in chains))   

In [ ]:
print("Percentage of samples with >10% breaks is {} and >0 is {}.".format(np.count_nonzero(sampleset3A.record.chain_break_fraction > 0.10)/numr*100, np.count_nonzero(sampleset3A.record.chain_break_fraction > 0.0)/numr*100))


In [ ]:
# Inspect Embedding
dwave.inspector.show(sampleset3A) 

## Chain Management for best Embedding

In [ ]:
# Choose best Embedding 
embedding = embedding3

### Adjust Chain Strength if necessary 

The following considerations and recommendations apply to chains.

- Prefer short chains to long chains.
- Prefer uniform chain lengths to uneven chains.
- Balance chain strength and problem range. Estimate chain strength and set just slightly above the minimum threshold needed, using strategies for auto-adjusting these chains. 

##### run again with each chain setting

In [ ]:
# Default
sampleset_default = FixedEmbeddingComposite(qpu, embedding).sample(bqm, 
                                                                   return_embedding=True, 
                                                                   answer_mode="raw", 
                                                                   num_reads=numr, 
                                                                   chain_strength= chnstr)


In [ ]:
analyze(sampleset_default, "default_chains", data)

You can set a chain strength relative to your problem’s largest bias by using, for example, the scaled() function.

In [ ]:
chnstr = scaled

In [ ]:
#from dwave.embedding.chain_strength import scaled
sampleset_scaled1 = FixedEmbeddingComposite(qpu, embedding).sample(bqm, 
                                                                   return_embedding=True, 
                                                                   answer_mode="raw", 
                                                                   num_reads=numr, 
                                                                   chain_strength= chnstr)  


In [ ]:
analyze(sampleset_scaled1, "scaled_chains1", data)

In [ ]:
#from dwave.embedding.chain_strength import scaled
sampleset_scaled2 = FixedEmbeddingComposite(qpu, embedding).sample(bqm, 
                                                                   return_embedding=True, 
                                                                   answer_mode="raw", 
                                                                   num_reads=numr, 
                                                                   chain_strength= chnstr)  


In [ ]:
analyze(sampleset_scaled2, "scaled_chains2", data)

Chain strength that attempts to compensate for torque that would break the chain.

In [ ]:
chnstr = uniform_torque_compensation

In [ ]:
#from dwave.embedding.chain_strength import uniform_torque_compensation
sampleset_torque1 = FixedEmbeddingComposite(qpu, embedding).sample(bqm, 
                                                                   return_embedding=True, 
                                                                   answer_mode="raw", 
                                                                   num_reads=numr, 
                                                                   chain_strength= chnstr)  


In [ ]:
analyze(sampleset_torque1, "torque_chains1", data)

In [ ]:
#from dwave.embedding.chain_strength import uniform_torque_compensation
sampleset_torque2 = FixedEmbeddingComposite(qpu, embedding).sample(bqm, 
                                                                   return_embedding=True, 
                                                                   answer_mode="raw", 
                                                                   num_reads=numr, 
                                                                   chain_strength= chnstr)  


In [ ]:
analyze(sampleset_torque2, "torque_chains2", data)

## Run for different Times and Schedules

In [ ]:
# With optimal chain strengths:
chnstr = uniform_torque_compensation

https://docs.dwavesys.com/docs/latest/handbook_qpu.html?highlight=anneal_schedule#annealing-schedule

https://docs.dwavesys.com/docs/latest/c_qpu_0.html?highlight=quench

In [ ]:
#qpu.properties["default_annealing_time"]      
# 20

### Find Sweet Spot for Annealing Time

In [ ]:
#20
sampleset_20 = FixedEmbeddingComposite(qpu, embedding).sample(bqm,
                                                              return_embedding=True, 
                                                              answer_mode="raw", 
                                                              num_reads=numr, 
                                                              chain_strength= chnstr,
                                                              annealing_time=20)

In [ ]:
analyze(sampleset_20, "time_20", data)

In [ ]:
#25
sampleset_25 = FixedEmbeddingComposite(qpu, embedding).sample(bqm,
                                                              return_embedding=True, 
                                                              answer_mode="raw", 
                                                              num_reads=numr, 
                                                              chain_strength= chnstr,
                                                              annealing_time=25)

In [ ]:
analyze(sampleset_25, "time_25", data)

In [ ]:
#30
sampleset_30 = FixedEmbeddingComposite(qpu, embedding).sample(bqm,
                                                              return_embedding=True, 
                                                              answer_mode="raw", 
                                                              num_reads=numr, 
                                                              chain_strength= chnstr,
                                                              annealing_time=30)

In [ ]:
analyze(sampleset_30, "time_30", data)

In [ ]:
#35
sampleset_35 = FixedEmbeddingComposite(qpu, embedding).sample(bqm,
                                                              return_embedding=True, 
                                                              answer_mode="raw", 
                                                              num_reads=numr, 
                                                              chain_strength= chnstr,
                                                              annealing_time=35)  


In [ ]:
analyze(sampleset_35, "time_35", data)

In [ ]:
#40
sampleset_40 = FixedEmbeddingComposite(qpu, embedding).sample(bqm,
                                                              return_embedding=True, 
                                                              answer_mode="raw", 
                                                              num_reads=numr, 
                                                              chain_strength= chnstr,
                                                              annealing_time=40)


In [ ]:
analyze(sampleset_40, "time_40", data)

In [ ]:
#45
sampleset_45 = FixedEmbeddingComposite(qpu, embedding).sample(bqm,
                                                              return_embedding=True, 
                                                              answer_mode="raw", 
                                                              num_reads=numr, 
                                                              chain_strength= chnstr,
                                                              annealing_time=45)


In [ ]:
analyze(sampleset_45, "time_45", data)

In [ ]:
#50
sampleset_50 = FixedEmbeddingComposite(qpu, embedding).sample(bqm,
                                                              return_embedding=True, 
                                                              answer_mode="raw", 
                                                              num_reads=numr, 
                                                              chain_strength= chnstr,
                                                              annealing_time=50)


In [ ]:
analyze(sampleset_50, "time_50", data)

### Find best Schedule

### Pause and Quench

<img src="https://docs.dwavesys.com/docs/latest/_images/16q-pause.png" alt="drawing" width="600"/>

First verify the quench schedule

##### run with best chain setting twice

In [ ]:
quench_schedule = [[0.0, 0.0], [30.0, 0.35], [230.0, 0.35], [250, 1.0]]

# [[t, s], [t_1, s_1], ..., [t_n, s_n]]
# Time t must increase for all points in the schedule.
# the anneal fraction s must start at s = 0.0 and end at s = 1.0

In [ ]:
sampleset_quench1 = FixedEmbeddingComposite(qpu, embedding).sample(bqm, 
                                                                  answer_mode="raw", 
                                                                  num_reads=numr, 
                                                                  chain_strength= chnstr,
                                                                  anneal_schedule=quench_schedule)  


In [ ]:
analyze(sampleset_quench1, "quench_schedule1", data)

In [ ]:
sampleset_quench2 = FixedEmbeddingComposite(qpu, embedding).sample(bqm, 
                                                                  answer_mode="raw", 
                                                                  num_reads=numr, 
                                                                  chain_strength= chnstr,
                                                                  anneal_schedule=quench_schedule)  


In [ ]:
analyze(sampleset_quench2, "quench_schedule2", data)

In [ ]:
quench_schedule = [[0.0, 0.0], [30.0, 0.40], [530.0, 0.40], [560, 1.0]]

# [[t, s], [t_1, s_1], ..., [t_n, s_n]]
# Time t must increase for all points in the schedule.
# the anneal fraction s must start at s = 0.0 and end at s = 1.0

In [ ]:
sampleset_quench3 = FixedEmbeddingComposite(qpu, embedding).sample(bqm, 
                                                                  answer_mode="raw", 
                                                                  num_reads=numr, 
                                                                  chain_strength= chnstr,
                                                                  anneal_schedule=quench_schedule)  


In [ ]:
analyze(sampleset_quench3, "quench_schedule3", data)

### Reverse

In brief, reverse annealing is a technique that makes it possible to refine known good local solutions, thereby increasing performance for certain applications.

There are three parameters you can use to configure reverse annealing using Ocean:

1. `anneal_schedule` defines the annealing schedule that should be followed.
2. `initial_state` specifies the classical state at which the reverse anneal should start.
3. `reinitialize_state` specifies whether or not the initial state should be used for every anneal in the request.  If False, then after the first, each subsequent anneal starts where the previous finished.

<img src="https://docs.dwavesys.com/docs/latest/_images/ra.png" alt="drawing" width="600"/>


A Reverse schedule always starts at `s = 1.0`, and ours reverses quickly to `s = 0.45`, pauses for `100 μs`, then quickly anneals forward. The schedule is formatted as a list of  **[time, s]** pairs. The next cell plots the schedule.

In [ ]:
# [[t, s], [t_1, s_1], ..., [t_n, s_n]]
# Time t must increase for all points in the schedule.
# the anneal fraction s must start and end at s = 1.0
max_slope = 1.0/qpu.properties["annealing_time_range"][0]
reverse_schedule = make_reverse_anneal_schedule(s_target=0.45, hold_time=180, ramp_up_slope=max_slope)
time_total = reverse_schedule[3][0]

print(reverse_schedule)
print("Total anneal-schedule time is {} us".format(time_total))

In [ ]:
make_anneal_plot(reverse_schedule)

In [ ]:
# Assign best Sampleset
best_sampleset = sampleset_quench3

In [ ]:

i5 = int(5.0/95*len(best_sampleset))  # Index i5 is about a 5% indexial move from the sample of lowest energy

initial = dict(zip(best_sampleset.variables, best_sampleset.record[i5].sample))
reverse_anneal_params = dict(anneal_schedule=reverse_schedule, initial_state=initial, reinitialize_state=False)

# The `reinitialize_state` parameter switches between two qualitatively different methods of local search 
# via quantum annealing.  With `reinitialize_state = 'True'`, many anneals are seeded from the single starting 
# state and the states returned, modulo time-dependent sources of error, are independently and identically 
# distributed.  With `reinitialize_state = 'False'`, the series of anneals behaves like a random walk and 
# is capable of exploring the solution space more broadly.  


In [ ]:
sampleset_reverse1 = FixedEmbeddingComposite(qpu, embedding).sample(bqm, 
                                                                   answer_mode="raw", 
                                                                   num_reads=numr, 
                                                                   chain_strength= chnstr,
                                                                   **reverse_anneal_params)  


In [ ]:
analyze(sampleset_reverse1, "reverse_schedule1", data)

In [ ]:
sampleset_reverse2 = FixedEmbeddingComposite(qpu, embedding).sample(bqm, 
                                                                   answer_mode="raw", 
                                                                   num_reads=numr, 
                                                                   chain_strength= chnstr,
                                                                   **reverse_anneal_params)  


In [ ]:
analyze(sampleset_reverse2, "reverse_schedule2", data)

#### Make table of solutions:

In [ ]:
df = pd.DataFrame(data, columns=["Label", "Best Known Solutions (%)", "Good Known Solutions (%)", "Lowest Energy", "Average Energy", "Standard Deviation"])
data

In [ ]:
df